# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096815


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:23,  2.97s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:00,  2.23s/it]

Rendering models:  13%|█▎        | 4/30 [00:09<01:02,  2.39s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:46,  1.87s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:23<00:01,  1.63it/s]

Rendering models:  93%|█████████▎| 28/30 [00:24<00:01,  1.62it/s]

Rendering models:  97%|█████████▋| 29/30 [00:25<00:00,  1.61it/s]

Rendering models: 100%|██████████| 30/30 [00:25<00:00,  1.61it/s]

not-logged-in-3af001450ab9bcf73c06    0.001283
Jnursssmith5263                       0.000904
not-logged-in-f696328c08d99f1bed52    0.246181
Alexander00                           0.001222
nurenai                               0.000498
wz19980805                            0.000616
Jonas_Cross                           0.000458
jnarayanbvg                           0.003033
not-logged-in-db23dc032fc427a4e4d9    0.000574
ferlin1998                            0.001721
not-logged-in-5f30b2bb9b7f9d49da41    0.628305
leclercsimon                          0.000521
not-logged-in-4607991ac840eb02289e    0.000465
trey.barkley                          0.066312
clairedeu                             0.000618
jlj080498                             0.001080
Mr_DMan                               0.000891
Lavadude                              0.017986
jordanrushworth                       0.001896
Tessaje04                             0.066087
equidad1                              0.000491
wgoltz       

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())